# Notebook pro hledání hyperparametrů - kostra

Několik pokusných běhů již proběhlo, každý ovšem následovala další úprava parametrů hledání. Výsledky budou nahrány až doběhnou. 

Zároveň bude zřejmě vhodné hledání rozšířit i na další variace v notebocích CIFAR10 a CIFAR100.
Momentálně se prohledávání provádí pouze nad datasetem CIFAR10 a náhodně inicializovaným mobilenetem s destilací a bez. Využíváno je předpočítaných logitů ze sešitu precompute_logits.

In [1]:
%pip install transformers[torch] huggingface_hub datasets evaluate torchvision optuna

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import knihoven a definice metod

In [2]:
from transformers import Trainer, TrainingArguments, MobileNetV2Config, MobileNetV2ForImageClassification, EarlyStoppingCallback
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
import numpy as np
import evaluate
import random
import pickle
import optuna
import torch
import os 

Resetování náhodného seedu pro replikovatelnost výsledků.
Zřejmě je možné části odebrat.

TODO: Odebrat zbytečná nastavení.

In [3]:
def reset_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

Nový wrapper, který pracuje přímo se soubory staženého a upraveného datasetu CIFAR10.
Využití načtení pomocí metody jako dříve není možné kvůli jiné checksum. 

Zároveň se již dotahují logity přímo z datasetu.

In [4]:
class CustomCIFAR10(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        self.data = []
        self.targets = []
        self.logits = []
        
        if self.train:
             for i in range(1, 6):
                 data_file = os.path.join(self.root, 'cifar-10-batches-py', f'data_batch_{i}')
                 with open(data_file, 'rb') as fo:
                     dict = pickle.load(fo, encoding='bytes')
                     self.data.append(dict[b'data'])
                     self.targets.extend(dict[b'labels'])
                     self.logits.extend(dict[b'logits'])  
        else:
            data_file = os.path.join(self.root, 'cifar-10-batches-py', 'test_batch')
            with open(data_file, 'rb') as fo:
                dict = pickle.load(fo, encoding='bytes')
                self.data.append(dict[b'data'])
                self.targets.extend(dict[b'labels'])
                self.logits.extend(dict[b'logits'])  

        self.data = np.concatenate(self.data, axis=0)
        self.targets = np.array(self.targets)
        self.logits = np.array(self.logits)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index].reshape(3, 32, 32).transpose(1, 2, 0)
        label = self.targets[index]
        logit = self.logits[index]
        
        image = Image.fromarray(image.astype('uint8'), 'RGB')
        logit = torch.tensor(logit, dtype=torch.float)
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)
            
        return {
            'pixel_values': image,
            'labels': label,
            'logits': logit
        }


Definice performance metrik.

In [5]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    accuracy = accuracy_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels)
    precision = precision_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')
    recall = recall_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')
    f1 = f1_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

Trénovací argumenty pro trainer. Třída rozšířena aby pojmula přímo temperature a parametr lambda. 

Tyto parametry tedy jsou přesunuty z traineru přímo do trénovacích parametrů, kde dávají větší smysl.

In [ ]:
class Custom_training_args(TrainingArguments):
    def __init__(self, lambda_param, temperature, *args, **kwargs):
        super().__init__(*args, **kwargs)    
        self.lambda_param = lambda_param
        self.temperature = temperature

In [6]:
def get_training_args(output_dir:str, logging_dir:str, remove_unused_columns:bool):
    return (
        Custom_training_args(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5, #Defaultní hodnota 
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=20,
        weight_decay=0.01,
        seed = 42,  #Defaultní hodnota 
        metric_for_best_model="f1",
        load_best_model_at_end=True,
        fp16=False, 
        logging_dir=logging_dir,
        remove_unused_columns=remove_unused_columns,
        lambda_param = 0.5, 
        temperature = 5
    ))

Náhodně inicializovaný MobileNetV2.

In [7]:
def get_random_init_mobilenet():
    reset_seed(42)
    student_config = MobileNetV2Config()
    student_config.num_labels = 10
    return MobileNetV2ForImageClassification(student_config)

In [10]:
reset_seed(42)

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test = CustomCIFAR10(root='../data/10-logits', train=False, transform=transform)
train = CustomCIFAR10(root='../data/10-logits', train=True, transform=transform)

### Hledání hyper parametrů náhodně inicializovaného modelu. 

In [13]:
training_args = get_training_args("./results/cifar10-random-search", './logs/cifar10-random-search', True)

Prozatím hledáme learning rate, batch size a weight decay.  

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3)
    }

Pro hledání je využito knihovny optuna, která má integraci s huggingface trainerem. Jako algoritmus je využit hyperband s doporučeným samplerem. 

Stále ještě dochází k ladění vhodné konfigurace ...

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=5, reduction_factor=4)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

Do traineru přidám callback s EarlyStoppingem ...

In [14]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

Hledáme na základě nejlepší metriky f1 score. V případě normálního (non-distilled) modelu by bylo možné využít i eval_loss. V případě distilled tréninku je ovšem jeho využití nevhodné, s rostoucím lambda parametrem bude "uměle" eval loss růst a tím pádem tyto konfigurace nebudou preferovány. 

Pro zachování jednoduchosti zvoleno v obou případech shodné nastavení na maximalizaci f1 score.

In [15]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    n_trials=80
)

Epoch,Training Loss,Validation Loss,Accuracy
1,1.999400,1.465344,0.461900
2,1.338000,1.238446,0.563300
3,1.135700,1.040728,0.626200
4,0.888400,0.949196,0.677800
5,0.773600,0.940272,0.678500
6,0.618400,0.835595,0.726800
7,0.558500,1.199692,0.655300
8,0.431500,1.572898,0.602600
9,0.356000,0.850076,0.743500
10,0.288900,0.897842,0.753100


TrainOutput(global_step=15640, training_loss=0.45282880482466326, metrics={'train_runtime': 5340.8309, 'train_samples_per_second': 187.237, 'train_steps_per_second': 2.928, 'total_flos': 2.020099608576e+18, 'train_loss': 0.45282880482466326, 'epoch': 20.0})

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

Dále došlo k úpravě s přijmáním model_init parametru a dotahováním nových hodnot temperature a lambda v případě hledání. Tato varianta zatím není kompatibilní s normálním tréninkem.


TODO: Doplnit IFs pro využitelnost i v normálním tréninku (model=not none, init_model = none).

In [18]:
class ImageDistilTrainer(Trainer):
    def __init__(self, model_init, *args, **kwargs):
        self.model_init = model_init
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
       
        super().__init__(model_init=model_init, *args, **kwargs)
        
        self.student = self.model_init()
        self.temperature = self.args.temperature
        self.lambda_param = self.args.lambda_param

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.student.to(device)


    def compute_loss(self, model, inputs, return_outputs=False):
        logits = inputs.pop("logits")

        student_output = model(**inputs)
        self.lambda_param = self.args.lambda_param
        self.temperature = self.args.temperature
        
        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        student_target_loss = student_output.loss

        loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss)
        return (loss, student_output) if return_outputs else loss

### Hledání hyper parametrů náhodně inicializovaného modelu s pomocí destilace znalostí 

Přidáno hledání nastavení temperature a lamda parametrů.

Teoreticky lze vyzkoušet hledání pouze temperature a lambda na základě výsledků předchozího hledání ...

In [19]:
reset_seed(42)

In [21]:
training_args = get_training_args("./results/cifar10-random-KD", './logs/cifar10-random-KD', False)

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }

In [22]:
trainer = ImageDistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=1, reduction_factor=4)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    n_trials=80
)